In [6]:
import numpy as np
import pandas as pd

from sklearn.neural_network import MLPRegressor
from sklearn.metrics import mean_squared_error
from sklearn.datasets import load_boston
from sklearn.model_selection import KFold, cross_val_score
from sklearn.model_selection import train_test_split

%matplotlib inline

import warnings
warnings.filterwarnings('ignore')

In [7]:
boston = load_boston()
X = pd.DataFrame(boston.data, columns=boston.feature_names)
y = boston.target
train_data, test_data, train_targets, test_targets = train_test_split(X, y, test_size=0.20, shuffle=True, random_state=42)

In [8]:
# Create tuples hidden_layer_sizes
hidden_layer_tuples = []
n_layers = [1, 2]
n_neurons = [10, 20, 30, 40, 50]
for layers in n_layers:
    for neurons in n_neurons:
        cur_size = (neurons,) * layers
        hidden_layer_tuples.append(cur_size)
print('The first hidden layer tuples:')
print(hidden_layer_tuples)

The first hidden layer tuples:
[(10,), (20,), (30,), (40,), (50,), (10, 10), (20, 20), (30, 30), (40, 40), (50, 50)]


In [9]:
def calculate_min(scores):
    test_scores = []
    min_test_score = np.inf
    for score in scores:
        if score < min_test_score:
            min_test_score = score
        test_scores.append(min_test_score)
    return test_scores

In [10]:
%%time
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint

all_val_scores = []
all_test_scores = []
n_iter = 100
num_folds=2
kf = KFold(n_splits=num_folds, shuffle=True, random_state=42)

for random_state in range(5):

    model = MLPRegressor(solver='adam', random_state=42)

    param_grid_rand = {
        'hidden_layer_sizes': hidden_layer_tuples,
        'activation': ['logistic', 'tanh', 'relu'],
        'alpha': np.random.uniform(0.00001, 0.99999, 1000),
        'batch_size': [*range(2, 300, 1)],
        'learning_rate': ['constant', 'invscaling', 'adaptive'],
        'learning_rate_init': np.random.uniform(0.00001, 0.99999, 1000),

    }

    reg = RandomizedSearchCV(model, param_grid_rand, n_iter = n_iter, scoring='neg_mean_squared_error', n_jobs=-1, cv=kf, verbose=2, random_state=random_state, return_train_score=True)

    reg.fit(train_data, train_targets)

    reg_test_score = mean_squared_error(test_targets, reg.predict(test_data))
    all_test_scores.append(reg_test_score)

    print("Best MSE {:.3f} params {}".format(-reg.best_score_, reg.best_params_))

    all_val_scores.append(calculate_min(-reg.cv_results_['mean_test_score']))


Fitting 2 folds for each of 100 candidates, totalling 200 fits
Best MSE 29.303 params {'learning_rate_init': 0.02596802901643288, 'learning_rate': 'adaptive', 'hidden_layer_sizes': (50, 50), 'batch_size': 243, 'alpha': 0.9855824340019916, 'activation': 'relu'}
Fitting 2 folds for each of 100 candidates, totalling 200 fits
Best MSE 32.656 params {'learning_rate_init': 0.008266334313569126, 'learning_rate': 'adaptive', 'hidden_layer_sizes': (30, 30), 'batch_size': 102, 'alpha': 0.2948996638130952, 'activation': 'logistic'}
Fitting 2 folds for each of 100 candidates, totalling 200 fits
Best MSE 32.004 params {'learning_rate_init': 0.056642272081697295, 'learning_rate': 'constant', 'hidden_layer_sizes': (20,), 'batch_size': 131, 'alpha': 0.01947282894169111, 'activation': 'relu'}
Fitting 2 folds for each of 100 candidates, totalling 200 fits
Best MSE 30.541 params {'learning_rate_init': 0.03227421068231593, 'learning_rate': 'adaptive', 'hidden_layer_sizes': (20, 20), 'batch_size': 209, 'al

In [11]:
df = pd.DataFrame(all_val_scores).transpose()
df

,0,1,2,3,4
0,315.933310,706.286312,93.339339,89.970649,87.252274
1,315.933310,706.286312,93.339339,81.995397,87.252274
2,87.171041,89.321150,90.789372,81.995397,87.252274
3,86.422520,89.321150,90.789372,60.169841,87.252274
4,86.422520,89.321150,90.789372,60.169841,86.599525
...,...,...,...,...,...
95,29.302909,32.655727,38.002481,30.540675,35.213431
96,29.302909,32.655727,38.002481,30.540675,35.213431
97,29.302909,32.655727,38.002481,30.540675,35.213431
98,29.302909,32.655727,32.004251,30.540675,35.213431


In [12]:
mean_std = []
for row_index in range(len(df)):
    row_np = df.iloc[row_index,:].to_numpy()
    mean_row = np.mean(row_np)
    std_row = np.std(row_np)
    mean_std.append((mean_row, std_row))
mean_std

[(258.5563767179493, 240.34022545843374),
 (256.9613264478938, 241.47754789378965),
 (87.30584698500475, 2.9805472868340446),
 (82.79103142822184, 11.414241588293999),
 (82.66048153651849, 11.36610111774175),
 (82.66048153651849, 11.36610111774175),
 (82.66048153651849, 11.36610111774175),
 (82.66048153651849, 11.36610111774175),
 (81.0356997554894, 10.454769611791326),
 (81.0356997554894, 10.454769611791326),
 (77.0493502388972, 11.612162083569634),
 (76.98006903879568, 11.553516526200239),
 (66.1819711194274, 19.776941242317495),
 (66.1819711194274, 19.776941242317495),
 (66.1819711194274, 19.776941242317495),
 (66.1819711194274, 19.776941242317495),
 (56.58082989637804, 19.17788896286269),
 (56.58082989637804, 19.17788896286269),
 (56.58082989637804, 19.17788896286269),
 (56.58082989637804, 19.17788896286269),
 (56.58082989637804, 19.17788896286269),
 (56.58082989637804, 19.17788896286269),
 (56.58082989637804, 19.17788896286269),
 (56.58082989637804, 19.17788896286269),
 (56.580829

In [13]:
final_df = pd.DataFrame(mean_std, columns=['Mean', 'Std'])
final_df

,Mean,Std
0,258.556377,240.340225
1,256.961326,241.477548
2,87.305847,2.980547
3,82.791031,11.414242
4,82.660482,11.366101
...,...,...
95,33.143045,3.151415
96,33.143045,3.151415
97,33.143045,3.151415
98,31.943399,2.007190


In [14]:
final_df.to_csv('Random  Search - Final Results.csv', index=False)

In [15]:
test_df = pd.DataFrame(all_test_scores, columns=['Test scores'])
test_df.to_csv('Random  Search - Test scores.csv', index=False)